In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import Fullscreen
#import seaborn
import holoviews as hv
import hvplot.pandas 
import panel as pn
import matplotlib as plt
%matplotlib inline
pn.extension('tabulator')
from panel.interact import interact
import param
import seaborn as sns
pn.extension(sizing_mode="stretch_width", server=True)

In [ ]:
#gdp = pd.read_csv("C:/Users/charl/OneDrive/Desktop/programming/panel_dashboard/species/data/gdp.csv",encoding="utf-8",delimiter=",",decimal=".")
cities = gpd.read_file(gpd.datasets.get_path("naturalearth_cities"))
food = pd.read_csv("C:/Users/charl/OneDrive/Desktop/programming/panel_dashboard/species/data/global-food.csv",encoding="utf-8",delimiter=",",decimal=".")
continents=gpd.read_file("C:/Users/charl/OneDrive/Desktop/programming/panel_dashboard/species/data/World_Continents.shp")

In [ ]:
continents_food = ['Europe', 'North America', 'South America', 'Asia', 'Africa', 'Australia'] #group by continent


In [ ]:
food_continents = food[food['Country'].isin(continents_food)]
food_continents.fillna(0)

In [ ]:
food_continents_interactive=food_continents


In [ ]:
# Radio buttons for  stats 
yaxis_food = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Production (t)','Yield (t/ha)','production__tonnes__per_capita'],
    button_type='success'
)

    
yaxis_food

In [ ]:
food_plot = food_continents_interactive.hvplot(x='Year', by='Country', y=yaxis_food, line_width=2, title="Food Production")

food_plot

In [ ]:
#bar plot 
food_filtered = food_continents_interactive[['Country', 'Year', 'Exports (t)', 'exports__tonnes__per_capita','Domestic supply (t)', 'domestic_supply__tonnes__per_capita','Imports (t)',
       'imports__tonnes__per_capita','Population' ]]

food_filtered_renamed=food_filtered.rename(columns={'Country':'Continent','Year':'Year','Exports (t)':'Exports(t)', 'exports__tonnes__per_capita':'Exports(t)_per_capita','Domestic supply (t)':'DomesticSupply(t)', 'domestic_supply__tonnes__per_capita':'DomesticSupply(t)_per_capita','Imports (t)':'Imports(t)','imports__tonnes__per_capita':'Imports(t)_per_capita' })

#food_bar_plot = food_continents_interactive.groupby('Country',type)
food_filtered_renamed.head()

In [ ]:
trade_stats = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Exports(t)', 'Imports(t)'], 
    button_type='success'
)

In [ ]:
trade_stats

In [ ]:
trade_stats_grouped= food_filtered_renamed.groupby(['Continent','Year']).sum()
#trade_stats_grouped.reset_index(inplace=True)


In [ ]:
trade_stats_grouped.head()

In [ ]:
#bar_plot = trade_stats_grouped(kind='bar',x='Continent',y='Exports(t)', 'Imports(t)', title='Import & Export Stats by year')

In [ ]:
plot_bars =trade_stats_grouped.hvplot(x='Year', by='Continent', y=trade_stats, line_width=2, title='Import & Export Stats per year and Continent')

plot_bars

In [ ]:
continents.head()

In [ ]:
merged_continents= continents.merge(food_continents, left_on='CONTINENT', right_on='Country')


In [ ]:
merged_continents.head()

In [ ]:

merged_filter=merged_continents[['CONTINENT', 'geometry','Year','Production (t)', 'Yield (t/ha)','production__tonnes__per_capita']]
#merged_years=merged_filter[merged_filter['Year'].isin(years)]
filtered_df = merged_filter[(merged_filter['Year'] >= 2000) & (merged_filter['Year'] <= 2022)]


In [ ]:
filtered_df.head() # filter per year for map
filtered_year_df = filtered_df[filtered_df['Year'] == 2020]


In [ ]:
stats = pn.widgets.Select(name='Production Stats', options=['Yield (t/ha)', 'production__tonnes__per_capita', 'area_harvested__ha__per_capita'])
stats

In [ ]:
m = filtered_year_df.explore('Yield (t/ha)', # show the Distance column
                   cmap='BuPu', # use the 'plasma' colormap from matplotlib
                   legend_kwds={'caption': 'Colormap based on Yield t(ha) for the year 2020'} # set the caption to a longer explanation
                  )
cities.explore(m=m,  color='grey')
m

In [ ]:
grouped_year=food_continents_interactive.groupby(['Country','Year']).sum()

grouped_year_interactive =grouped_year

In [ ]:
#scatter plot
x = pn.widgets.Select(name='Production Statistics', options=['Land Use (ha)', 'Yield (t/ha)', 'area_harvested__ha__per_capita'])
y = pn.widgets.Select(name='Food Supply Statistics', options=['Food (t)', 'food__tonnes__per_capita', 'Food supply (g per capita per day)'])

plot_scatter = grouped_year_interactive.hvplot(x=x, y=y, kind='scatter', colorbar=False, width=400,alpha=0.5,by="Country")

plot_scatter_layout = pn.Column(
    pn.pane.Markdown("### Select combinations of different variables grouped by year to show scatter plot", styles={'font-weight': 'bold', 'color':'grey'}),
    x,y, plot_scatter
    
)

In [ ]:
plot_scatter_layout

In [ ]:
@pn.cache  # gapminder data
def get_dataset():
    url = 'https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv'
    return pd.read_csv(url)

dataset = get_dataset()

YEARS = [int(year) for year in dataset.year.unique()]

#dataset.tail(10)

In [ ]:
#dashboard layout using Template
text="A simple dashboard to practice visualizations using the Panel Framework"
template = pn.template.FastListTemplate(
    title='Agricultural Production Statistics', 
    sidebar=[pn.pane.Markdown("# Dashboard showing agricultural production statistics by continent and year"),
             text,
             pn.pane.Markdown("#### Data from Our World in Data.https://ourworldindata.org"), 
              ],
    main=[
        pn.Row(
            pn.Column(
                yaxis_food,
                food_plot,
                sizing_mode="stretch_width"),
                plot_scatter_layout,
                sizing_mode="stretch_width"),
    
        
        pn.Row(
            pn.Column(
            pn.pane.plot.Folium(m, height=400),
            sizing_mode="stretch_width"),
            pn.Column(
            trade_stats,
            plot_bars,  
           
       
        ))
    ] ,   
            header_background="#6b8e23",

)
        
        
        
template.show()

